In [ ]:
%pip install -U langchain jq

In [ ]:
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["monster_source"] = record.get("monster_source")
    metadata["monster_name"] = record.get("monster_name")
    metadata["field"] = record.get("field")

    if "source" in metadata:
        source = metadata["source"].split("/")
        source = source[source.index("langchain"):]
        metadata["source"] = "/".join(source)

    return metadata

In [ ]:
from langchain.document_loaders import JSONLoader
from pprint import pprint

fields = ["trait", "action", "legendary", "bonus", "descriptions"]
documents_by_field = {}

for field in fields:
    jq_schema = f'.[].text'
    loader = JSONLoader(
        file_path='./monster_text.json', jq_schema=jq_schema, content_key=field, metadata_func=metadata_func)
    documents_by_field[field] = loader.load()

pprint(documents_by_field)
